In [ ]:
import fiona
from shapely.geometry import shape, mapping
from shapely.ops import transform
import pyproj
import folium
from branca.colormap import linear

# 좌표계 변환 (EPSG:5186 → WGS84)
project = pyproj.Transformer.from_crs("EPSG:5186", "EPSG:4326", always_xy=True).transform

# SHP 파일 경로
shp_path = "/home/gmlwls5168/code/gongmo/src/notebooks/nijin/BND_ADM_DONG_PG/BND_ADM_DONG_PG.shp"

# 이름 매핑 (SHP 행정동 이름 → 점수 dictionary 이름)
name_mapping = {
    "광장동": "광장동", "구의1동": "구의제1동", "구의2동": "구의제2동", "구의3동": "구의제3동",
    "군자동": "군자동", "능동": "능동", "자양1동": "자양제1동", "자양2동": "자양제2동",
    "자양3동": "자양제3동", "자양4동": "자양제4동", "중곡1동": "중곡제1동", "중곡2동": "중곡제2동",
    "중곡3동": "중곡제3동", "중곡4동": "중곡제4동", "화양동": "화양동"
}

# 위험도 점수
risk_dict = {
    "화양동": 1.000, "중곡제4동": 0.774, "구의제1동": 0.756, "자양제2동": 0.701,
    "자양제1동": 0.595, "자양제4동": 0.563, "광장동": 0.544, "구의제2동": 0.535,
    "군자동": 0.516, "중곡제1동": 0.513, "중곡제2동": 0.500, "중곡제3동": 0.480,
    "구의제3동": 0.470, "자양제3동": 0.450
}

# SHP 로딩 및 좌표계 변환
features = []
with fiona.open(shp_path, encoding="cp949") as src:
    for feat in src:
        adm_nm = feat["properties"]["ADM_NM"].strip()
        if adm_nm in name_mapping:
            mapped_name = name_mapping[adm_nm]
            geom = shape(feat["geometry"])
            geom_wgs84 = transform(project, geom)
            features.append({
                "type": "Feature",
                "geometry": mapping(geom_wgs84),
                "properties": {"name": mapped_name}
            })

# GeoJSON 생성
geojson = {"type": "FeatureCollection", "features": features}

# 지도 객체 생성 (위치 조정)
m = folium.Map(location=[37.552, 127.085], zoom_start=14)

# 색상 스타일 정의
colormap = linear.Reds_09.scale(0, 1)
colormap.caption = "광진구 행정동별 취약계층 위험도"

# 스타일 함수 (투명도 조정)
def style_function(feature):
    name = feature["properties"]["name"]
    score = risk_dict.get(name, 0)
    return {
        "fillColor": colormap(score),
        "color": "black",
        "weight": 1.5,
        "fillOpacity": 0.5  # ✅ 투명도 조정됨
    }

# GeoJson 추가
folium.GeoJson(
    geojson,
    name="광진구 위험도 지도",
    style_function=style_function,
    tooltip=folium.GeoJsonTooltip(fields=["name"], aliases=["행정동:"])
).add_to(m)

colormap.add_to(m)

In [ ]:
import geopandas as gpd
import folium
from branca.colormap import linear

# ✅ 정확한 SHP 경로
shp_path = "/home/gmlwls5168/code/gongmo/src/notebooks/nijin/BND_ADM_DONG_PG/BND_ADM_DONG_PG.shp"

# 이름 매핑
name_mapping = {
    "광장동": "광장동", "구의1동": "구의제1동", "구의2동": "구의제2동", "구의3동": "구의제3동",
    "군자동": "군자동", "능동": "능동", "자양1동": "자양제1동", "자양2동": "자양제2동",
    "자양3동": "자양제3동", "자양4동": "자양제4동", "중곡1동": "중곡제1동", "중곡2동": "중곡제2동",
    "중곡3동": "중곡제3동", "중곡4동": "중곡제4동", "화양동": "화양동"
}

# 데이터 로드 및 변환
gdf = gpd.read_file(shp_path, encoding="cp949")
gdf = gdf[gdf["ADM_NM"].isin(name_mapping.keys())]
gdf = gdf.to_crs(epsg=4326)
gdf["행정동"] = gdf["ADM_NM"].map(name_mapping)

# 예시 비율 데이터
gdf["여성비율"] = gdf["행정동"].map({
    "광장동": 0.50, "구의제1동": 0.52, "구의제2동": 0.55, "구의제3동": 0.49,
    "군자동": 0.53, "능동": 0.54, "자양제1동": 0.51, "자양제2동": 0.56,
    "자양제3동": 0.50, "자양제4동": 0.52, "중곡제1동": 0.51, "중곡제2동": 0.53,
    "중곡제3동": 0.50, "중곡제4동": 0.48, "화양동": 0.57
})
gdf["1인가구비율"] = gdf["행정동"].map({
    "광장동": 0.35, "구의제1동": 0.38, "구의제2동": 0.40, "구의제3동": 0.32,
    "군자동": 0.36, "능동": 0.33, "자양제1동": 0.37, "자양제2동": 0.42,
    "자양제3동": 0.34, "자양제4동": 0.36, "중곡제1동": 0.30, "중곡제2동": 0.31,
    "중곡제3동": 0.29, "중곡제4동": 0.28, "화양동": 0.45
})
gdf["노인비율"] = gdf["행정동"].map({
    "광장동": 0.12, "구의제1동": 0.14, "구의제2동": 0.13, "구의제3동": 0.10,
    "군자동": 0.15, "능동": 0.16, "자양제1동": 0.11, "자양제2동": 0.09,
    "자양제3동": 0.13, "자양제4동": 0.14, "중곡제1동": 0.20, "중곡제2동": 0.22,
    "중곡제3동": 0.21, "중곡제4동": 0.19, "화양동": 0.08
})

# 공통 시각화 함수
def make_map(column, caption, filename):
    m = folium.Map(location=[37.552, 127.085], zoom_start=14)
    cmap = linear.Reds_09.scale(gdf[column].min(), gdf[column].max())
    cmap.caption = caption

    folium.GeoJson(
        gdf,
        style_function=lambda feat: {
            "fillColor": cmap(feat["properties"][column]),
            "color": "black",
            "weight": 1.5,
            "fillOpacity": 0.5,
        },
        tooltip=folium.GeoJsonTooltip(fields=["행정동", column], aliases=["행정동:", caption])
    ).add_to(m)
    cmap.add_to(m)
    m.save(filename)
    print(f"✅ 저장 완료: {filename}")

# 1. 여성 비율
make_map("여성비율", "여성 인구 비율", "gwangjin_female_ratio_map.html")

# 2. 1인가구 비율
make_map("1인가구비율", "1인가구 비율", "gwangjin_single_ratio_map.html")

# 3. 노인 비율
make_map("노인비율", "노인 인구 비율", "gwangjin_elderly_ratio_map.html")

In [4]:
import fiona
from shapely.geometry import shape, mapping
from shapely.ops import transform
import pyproj
import pandas as pd
import folium
from branca.colormap import linear

# -------------------- 1. 좌표계 변환 정의 --------------------
project = pyproj.Transformer.from_crs("EPSG:5186", "EPSG:4326", always_xy=True).transform

# -------------------- 2. 파일 경로 정의 --------------------
shp_path = "/home/gmlwls5168/code/gongmo/src/notebooks/nijin/BND_ADM_DONG_PG/BND_ADM_DONG_PG.shp"         # SHP 파일 (필수: .shp, .shx, .dbf 같이 있어야 함)
cctv_path = "/home/gmlwls5168/code/gongmo/src/data/proceeded_cctv.csv"         # CCTV 데이터
lamppost_path = "/home/gmlwls5168/code/gongmo/src/data/stat_xy.csv"            # 보행등 데이터

# -------------------- 3. 이름 매핑 및 1인가구 비율 --------------------
name_mapping = {
    "광장동": "광장동", "구의1동": "구의제1동", "구의2동": "구의제2동", "구의3동": "구의제3동",
    "군자동": "군자동", "능동": "능동", "자양1동": "자양제1동", "자양2동": "자양제2동",
    "자양3동": "자양제3동", "자양4동": "자양제4동", "중곡1동": "중곡제1동", "중곡2동": "중곡제2동",
    "중곡3동": "중곡제3동", "중곡4동": "중곡제4동", "화양동": "화양동"
}
single_ratio = {
    "광장동": 0.35, "구의제1동": 0.38, "구의제2동": 0.40, "구의제3동": 0.32,
    "군자동": 0.36, "능동": 0.33, "자양제1동": 0.37, "자양제2동": 0.42,
    "자양제3동": 0.34, "자양제4동": 0.36, "중곡제1동": 0.30, "중곡제2동": 0.31,
    "중곡제3동": 0.29, "중곡제4동": 0.28, "화양동": 0.45
}

# -------------------- 4. SHP → GeoJSON 변환 --------------------
features = []
with fiona.open(shp_path, encoding="cp949") as src:
    for feat in src:
        adm_nm = feat["properties"]["ADM_NM"].strip()
        if adm_nm in name_mapping:
            mapped_name = name_mapping[adm_nm]
            geom = shape(feat["geometry"])
            geom_wgs84 = transform(project, geom)
            features.append({
                "type": "Feature",
                "geometry": mapping(geom_wgs84),
                "properties": {
                    "행정동": mapped_name,
                    "1인가구비율": single_ratio[mapped_name]
                }
            })

geojson = {"type": "FeatureCollection", "features": features}

# -------------------- 5. CCTV & 보행등 데이터 로딩 --------------------
cctv_df = pd.read_csv(cctv_path, encoding="utf-8")
lamppost_df = pd.read_csv(lamppost_path, encoding="cp949")
lamppost_gwangjin = lamppost_df[lamppost_df["자치구"] == "광진구"]

# -------------------- 6. folium 지도 생성 --------------------
m = folium.Map(location=[37.552, 127.085], zoom_start=14)
cmap = linear.Reds_09.scale(min(single_ratio.values()), max(single_ratio.values()))
cmap.caption = "1인가구 비율"

# GeoJson 시각화
folium.GeoJson(
    geojson,
    style_function=lambda feat: {
        "fillColor": cmap(feat["properties"]["1인가구비율"]),
        "color": "black",
        "weight": 1.5,
        "fillOpacity": 0.5
    },
    tooltip=folium.GeoJsonTooltip(fields=["행정동", "1인가구비율"], aliases=["행정동:", "1인가구 비율"])
).add_to(m)

# CCTV (파란색)
for _, row in cctv_df.iterrows():
    folium.CircleMarker(
        location=[row["위도"], row["경도"]],
        radius=2.5,
        color="blue",
        fill=True,
        fill_opacity=0.7,
        tooltip="CCTV"
    ).add_to(m)

# 보행등 (빨간색)
for _, row in lamppost_gwangjin.iterrows():
    folium.CircleMarker(
        location=[row["위도"], row["경도"]],
        radius=2.5,
        color="red",
        fill=True,
        fill_opacity=0.8,
        tooltip="보행등"
    ).add_to(m)

# 컬러맵 추가
cmap.add_to(m)

# -------------------- 7. 지도 보기 또는 저장 --------------------
m  # JupyterLab에서는 이 한 줄로 지도 출력 가능
# m.save("gwangjin_single_ratio_with_lights.html")  # 저장 원할 경우
